GROUP MEMBERS
NAKATE AUDREY             2100700695 21/U/0695
OLOO WILLIAM              2100719370 21/U/19370/PS
NAGGAYI LYTHN KIBIRIGE    2100723414 21/U/23414/PS
KIRABO NAKALEMBE PATRICIA 2100711033 21/U/11033/PS
SEMPIJJA IBRAHIM          2100712598 21/U/12598/PS

We are building models that predict the points wine will get based on the rest of the features of the dataset i.e. descripiton, designation, points, price and taster_name among other features.

In [1]:
import pandas as pd
from sklearn.metrics import accuracy_score

In [2]:
df = pd.read_csv("wine-reviews.csv", index_col=0)
df = df.dropna() #dropping all rows with missing data for any of the features.
print(df.shape)
df.head()

(22387, 13)


,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery
4,US,"Much like the regular bottling from 2012, this...",Vintner's Reserve Wild Child Block,87,65.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks
10,US,"Soft, supple plum envelopes an oaky structure ...",Mountain Cuvée,87,19.0,California,Napa Valley,Napa,Virginie Boone,@vboone,Kirkland Signature 2011 Mountain Cuvée Caberne...,Cabernet Sauvignon,Kirkland Signature
23,US,This wine from the Geneseo district offers aro...,Signature Selection,87,22.0,California,Paso Robles,Central Coast,Matt Kettmann,@mattkettmann,Bianchi 2011 Signature Selection Merlot (Paso ...,Merlot,Bianchi
25,US,Oak and earth intermingle around robust aromas...,King Ridge Vineyard,87,69.0,California,Sonoma Coast,Sonoma,Virginie Boone,@vboone,Castello di Amorosa 2011 King Ridge Vineyard P...,Pinot Noir,Castello di Amorosa
35,US,As with many of the Erath 2010 vineyard design...,Hyland,86,50.0,Oregon,McMinnville,Willamette Valley,Paul Gregutt,@paulgwine,Erath 2010 Hyland Pinot Noir (McMinnville),Pinot Noir,Erath


TRANSFORMING ALL TEXT DATA MINUS THE DESCRIPTION COLUMNS TO THEIR NUMERICAL EQUIVALENTS

In [3]:
#Transforming all text data columns to numerical equivalents
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
df["country"] = label_encoder.fit_transform(df["country"])
df["designation"] = label_encoder.fit_transform(df["designation"])
df["province"] = label_encoder.fit_transform(df["province"])
df["region_1"] = label_encoder.fit_transform(df["region_1"])
df["region_2"] = label_encoder.fit_transform(df["region_2"])
df["taster_name"] = label_encoder.fit_transform(df["taster_name"])
df["taster_twitter_handle"] = label_encoder.fit_transform(df["taster_twitter_handle"])
df["title"] = label_encoder.fit_transform(df["title"])
df["variety"] = label_encoder.fit_transform(df["variety"])
df["winery"] = label_encoder.fit_transform(df["winery"])

In [4]:
df.head()

,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery
4,0,"Much like the regular bottling from 2012, this...",8991,87,65.0,2,164,16,6,4,17492,111,2368
10,0,"Soft, supple plum envelopes an oaky structure ...",5382,87,19.0,0,82,6,10,6,9958,19,1343
23,0,This wine from the Geneseo district offers aro...,7415,87,22.0,0,93,1,4,3,2114,74,274
25,0,Oak and earth intermingle around robust aromas...,4234,87,69.0,0,132,12,10,6,3424,111,462
35,0,As with many of the Erath 2010 vineyard design...,3866,86,50.0,2,67,16,6,4,6528,111,851


PREPROCESSING THE TEXT DATA TO REMOVE UNNCESSARY WORDS AND SYMBOLS AND TO CHANGE ALL THE LETTERS TO LOWERCASE

In [5]:
#remove less important words from the description for purposes of classification
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
for i in df.description:
    tokens = word_tokenize(i)
    # Changing all the letters to lowercase 
    tokens_low = [w.lower() for w in tokens]
    # Removing all non-alphabetics from the descriptions 
    words = [word for word in tokens_low if word.isalpha()]
    #Removing stopwords
    stopwords = nltk.corpus.stopwords.words('english')
    moreStopwords = ["drink" , 'now', 'wine' ,'flavour']
    stopwords.extend(moreStopwords)
    filter_sen = [w for w in words if not w in stopwords]
    #Using stemming for normalisation
    porter  = PorterStemmer()
    stemmed = [porter.stem(word) for word in filter_sen]
    #to join the words into sentences again and replace them with their originals in the dataframe
    sentence = " ".join(w for w in stemmed)
    df = df.replace(i , sentence)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


APPLYING TfidVectorizer TO THE TEXT DATA

In [6]:
#applying TfidVectorizer to the text data
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(stop_words='english', max_df=0.7)
description_vectorized =  vectorizer.fit_transform(df["description"].values.astype('U'))
df["description"] = description_vectorized.toarray()

SPLITTING THE DATA INTO TRAINING AND TEST SETS

In [7]:
#split into training and test data
from sklearn.model_selection import train_test_split
target = df['points']
df.drop(['points'], axis=1)
X_train, X_test, y_train, y_test = train_test_split(df, target, test_size = 0.3, random_state=100)
print("The total training data is {}".format(X_train.shape))
print("The total test data is {}".format(X_test.shape))

The total training data is (15670, 13)
The total test data is (6717, 13)


SCALING THE FEATURES
This will ensure that features with larger values do not affect the prediction disproportionately compared to the smaller values

In [8]:

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

THE SUPPORT VECTOR MACHINE MODEL

In [9]:
#ALGORITHM 1 - SVM
#using the support vector machine algorithm with linear approach to build the model on the testing and training data
from sklearn.svm import SVC

svm_classifier = SVC(kernel='linear', random_state=100)  
#training the model
svm_classifier.fit(X_train, y_train)
#testing the model
svm_y_pred = svm_classifier.predict(X_test)
#Calculating the accuraccy score of the algorithm
100* accuracy_score(y_test, svm_y_pred)

99.94044960547863

THE LOGISTIC REGRESSION MODEL

In [10]:
#ALGORITHM 2 - LOGISTIC REGRESSION
from sklearn.linear_model import LogisticRegression

lr_classifier = LogisticRegression(max_iter=1000)
lr_classifier.fit(X_train, y_train)

#predicting the test set results
lr_y_pred = lr_classifier.predict(X_test)
#Calculating the accuracy of the algorithm
100* accuracy_score(y_test, lr_y_pred)

91.27586720262022

THE NAIVE BAYES MODEL

In [11]:
#A probabilistic classifier based on Bayes theorem used to classify the data
from sklearn.naive_bayes import GaussianNB
model = GaussianNB()
model.fit(X_train,y_train)

#predicting the test set results
nb_y_pred = model.predict(X_test)
#Calculating the accuraacy of the algorithm
100* accuracy_score(y_test, nb_y_pred)

99.95533720410897

Since the naive bayes algorithm results in the highest accuracy when predicting, we shall tune the naive bayes model using grid search

In [15]:
import numpy as np
from sklearn.model_selection import GridSearchCV

param_grid_np = {
    "var_smoothing": np.logspace(0, -9, num=100)
}

nbModel_grid = GridSearchCV(estimator=model, param_grid=param_grid_np, verbose=1, cv=10, n_jobs=-1)
nbModel_grid.fit(X_train, y_train)
print(nbModel_grid.best_estimator_)
#predicting the test set results
y_pred = nbModel_grid.best_estimator_.predict(X_test)
#calculating the accuracy of the tuned naive bayes model
100* accuracy_score(y_test, y_pred)

Fitting 10 folds for each of 100 candidates, totalling 1000 fits


c:\Users\Admin\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:700: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=10.
  warnings.warn(


GaussianNB(var_smoothing=3.511191734215127e-05)


99.94044960547863